In [ ]:
import nb_loader

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from pynq_dpu import DpuOverlay

In [ ]:
from cam import Camera
cam = Camera()

In [ ]:
rgb = cam.get_rgb()
plt.imshow(rgb)
plt.show()

In [ ]:
overlay = DpuOverlay("dpu.bit")
overlay.load_model("dpu_tf_inceptionv1.xmodel")

In [ ]:
dpu = overlay.runner

inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()

shapeIn = tuple(inputTensors[0].dims)
shapeOut = tuple(outputTensors[0].dims)
outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])

softmax = np.empty(outputSize)

In [ ]:
output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
image = input_data[0]

In [ ]:
import preproc as pre
import postproc as post

In [ ]:
def run(img, display=False):
    preprocessed = pre.process(img)
    image[0,...] = preprocessed.reshape(shapeIn[1:])
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    temp = [j.reshape(1, outputSize) for j in output_data]
    softmax = post.calculate_softmax(temp[0][0])
    if display:
        _, ax = plt.subplots(1)
        _ = ax.imshow(img)
        print("Classification: {}".format(post.predict_label(softmax)))

In [ ]:
for i in range(0, 10):
    run(cam.get_rgb(), display=True)

In [ ]:
cam.close()